## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-21-16-48-25 +0000,nyt,Security Officials Are Treating Charlie Kirk’s...,https://www.nytimes.com/2025/09/21/us/politics...
1,2025-09-21-16-48-14 +0000,nypost,LI’s Mike DelGuidice went from living in a tra...,https://nypost.com/2025/09/21/us-news/lis-mike...
2,2025-09-21-16-47-10 +0000,nyt,Thousands Flock to Phoenix for Charlie Kirk’s ...,https://www.nytimes.com/2025/09/20/us/politics...
3,2025-09-21-16-46-00 +0000,wsj,Huge Crowds Expected at Charlie Kirk Memorial,https://www.wsj.com/us-news/charlie-kirk-funer...
4,2025-09-21-16-43-26 +0000,startribune,"Live: Christian Darrisaw, Harrison Smith retur...",https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
61,trump,32
3,kirk,17
2,charlie,14
74,palestinian,11
53,will,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-09-21-16-42-16 +0000,wapo,Live updates: Charlie Kirk memorial to draw Tr...,https://www.washingtonpost.com/politics/2025/0...,84
165,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...,72
140,2025-09-20-23-11-48 +0000,cbc,White House scrambles to clarify Trump plan to...,https://www.cbc.ca/news/world/trump-travel-ban...,63
120,2025-09-21-02-03-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...,62
126,2025-09-21-01-22-42 +0000,nypost,Elon Musk resurfaces Harris tweet calling for ...,https://nypost.com/2025/09/20/us-news/elon-mus...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,84,2025-09-21-16-42-16 +0000,wapo,Live updates: Charlie Kirk memorial to draw Tr...,https://www.washingtonpost.com/politics/2025/0...
163,55,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...
41,51,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
70,43,2025-09-21-10-13-45 +0000,nypost,Crazed gunman shouted ‘Free Palestine’ before ...,https://nypost.com/2025/09/21/us-news/alleged-...
120,30,2025-09-21-02-03-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
5,27,2025-09-21-16-43-08 +0000,nyt,Jimmy Kimmel News Stirs Up Conservative Podcas...,https://www.nytimes.com/2025/09/20/style/the-m...
158,26,2025-09-20-20-44-23 +0000,nypost,Chuck Schumer and Hakeem Jeffries ‘demand’ mee...,https://nypost.com/2025/09/20/us-news/chuck-sc...
109,26,2025-09-21-05-01-20 +0000,bbc,"Israel will occupy more West Bank land, but re...",https://www.bbc.com/news/articles/c0ez9qxzl2jo...
59,24,2025-09-21-11-37-51 +0000,nypost,Erika Kirk reveals she wears husband Charlie’s...,https://nypost.com/2025/09/21/us-news/erika-ki...
105,22,2025-09-21-06-59-08 +0000,nypost,"Sean Duffy warns Chicago, Boston transit syste...",https://nypost.com/2025/09/21/us-news/sean-duf...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
